![imagenes](logo.png)

In [5]:
import pandas as pd
import numpy as np
from siuba import *

In [55]:
concentrado = {"Nombre":["José","Miguel","Andrea"],
               "Matrícula":[123,456,789],
               "Matemáticas":[10,9,10],
               "Español":[6,5,5],
               "Lectura":[5,10,6],
               "Aprov_trim":["Sí","Sí","No"],
               "Aprov_gral":["Sí","No","Sí"]    
}

concentrado = pd.DataFrame(concentrado)
concentrado

,Nombre,Matrícula,Matemáticas,Español,Lectura,Aprov_trim,Aprov_gral
0,José,123,10,6,5,Sí,Sí
1,Miguel,456,9,5,10,Sí,No
2,Andrea,789,10,5,6,No,Sí


## Datos en formato tidy

|Nombre | Matrícula|Materia     | Evaluación|Aprovechamiento |Resultado |
|:------|---------:|:-----------|----------:|:---------------|:---------|
|José   |       123|Matemáticas |         10|Aprov_trim      |Sí        |
|José   |       123|Matemáticas |         10|Aprov_gral      |Sí        |
|José   |       123|Español     |          6|Aprov_trim      |Sí        |
|José   |       123|Español     |          6|Aprov_gral      |Sí        |
|José   |       123|Lectura     |          5|Aprov_trim      |Sí        |
|José   |       123|Lectura     |          5|Aprov_gral      |Sí        |
|Miguel |       456|Matemáticas |          9|Aprov_trim      |Sí        |
|Miguel |       456|Matemáticas |          9|Aprov_gral      |No        |
|Miguel |       456|Español     |          5|Aprov_trim      |Sí        |
|Miguel |       456|Español     |          5|Aprov_gral      |No        |
|Miguel |       456|Lectura     |         10|Aprov_trim      |Sí        |
|Miguel |       456|Lectura     |         10|Aprov_gral      |No        |
|Andrea |       789|Matemáticas |         10|Aprov_trim      |No        |
|Andrea |       789|Matemáticas |         10|Aprov_gral      |Sí        |
|Andrea |       789|Español     |          5|Aprov_trim      |No        |
|Andrea |       789|Español     |          5|Aprov_gral      |Sí        |
|Andrea |       789|Lectura     |          6|Aprov_trim      |No        |
|Andrea |       789|Lectura     |          6|Aprov_gral      |Sí        |

In [56]:
def extractor(x):
    auxiliar = (concentrado >> 
                select(_.Nombre,_.Matrícula, _.Evaluación == x) >>
                mutate(Materia = x)
               )
    return auxiliar  

In [57]:
extractor("Matemáticas")

,Nombre,Matrícula,Evaluación,Materia
0,José,123,10,Matemáticas
1,Miguel,456,9,Matemáticas
2,Andrea,789,10,Matemáticas


In [58]:
previo = pd.concat([extractor(x) for x in concentrado.columns[2:5]])
previo

,Nombre,Matrícula,Evaluación,Materia
0,José,123,10,Matemáticas
1,Miguel,456,9,Matemáticas
2,Andrea,789,10,Matemáticas
0,José,123,6,Español
1,Miguel,456,5,Español
2,Andrea,789,5,Español
0,José,123,5,Lectura
1,Miguel,456,10,Lectura
2,Andrea,789,6,Lectura


In [75]:
print(concentrado >> gather("Materia",
                      "Evaluación",
                      _[2:5]) >>
               gather("Aprovechamiento",
                      "Resultado",
                       _[2:4]) >>
               group_by(_.Materia) >>
               filter(_.Evaluación == _.Evaluación.max(),_.Aprovechamiento == "Aprov_trim") >>
 ungroup()
    )

   Nombre  Matrícula      Materia  Evaluación Aprovechamiento Resultado
0    José        123  Matemáticas          10      Aprov_trim        Sí
1  Andrea        789  Matemáticas          10      Aprov_trim        No
2    José        123      Español           6      Aprov_trim        Sí
3  Miguel        456      Lectura          10      Aprov_trim        Sí
